In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.multivariate.manova import MANOVA
from dfply import *
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.multivariate.manova import MANOVA

In [3]:
# Analyse + fix missing values
data = pd.read_csv('prepared_data.csv', sep='\t', index_col=0)
print(data.isna().sum())
pd.set_option('use_inf_as_na',True)
data = data.dropna(how='any')

print(data.isna().sum())

error_rate           0
avg_char_duration    0
screen_size          0
swipe_finger         1
dtype: int64
error_rate           0
avg_char_duration    0
screen_size          0
swipe_finger         0
dtype: int64


## Visualisation of data befor removing outliers

## Removing outliers per category

In [4]:

def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

data = remove_outlier(data, 'screen_size')
#data[data["error_rate"] > 1] 
data

,error_rate,avg_char_duration,screen_size,swipe_finger
uid,,,,
003t6nt1ppo92gsagrn9op9mjg,2.875000,566.518056,273600,Thumb
0069j4rbe1si9pjqgfhks8sprm,0.074074,451270.329527,339076,Thumb
00h10g2j0m7vujuscdvispdkub,0.170370,506.169809,280800,Index
01rg5cvptn05eag69m0gckjer9,0.200000,300.044823,370944,Thumb
038qovd34aj82cqc9ip8fqu6sc,0.101449,346.902783,230400,Thumb
...,...,...,...,...
vs9nmoj0kboa0uhoubblculnc7,0.045455,141.184040,370944,Thumb
vs9o5ggaa18pakfr7j5p7p9rj9,0.239437,259.976649,230400,Index
vtc9pr98mr2gs793s50jg9g15m,0.077465,288.038450,230400,Thumb


In [5]:
data_small = data[data['screen_size'] < 280000]
data_big = data[data['screen_size'] >= 280000]


data_small.loc[data_small["screen_size"] > 0, 'screen_size'] = "Small"
data_big.loc[data_big["screen_size"] > 0, 'screen_size'] = "Big"

data_merged = data_small.merge(data_big, how = 'outer')
data_merged

,error_rate,avg_char_duration,screen_size,swipe_finger
0,2.875000,566.518056,Small,Thumb
1,0.101449,346.902783,Small,Thumb
2,0.052174,200.908248,Small,Thumb
3,0.463768,223.830191,Small,Index
4,0.086420,251.817642,Small,Thumb
...,...,...,...,...
1299,20.900000,38.115741,Big,Thumb
1300,0.400000,451.528427,Big,Index
1301,0.080000,427.315869,Big,Thumb
1302,0.100719,149.987698,Big,Index


In [6]:
#split in categories
data_small_thumb = data_merged[(data_merged['screen_size'] == "Small")&(data_merged['swipe_finger'] == "Thumb")]
data_small_index = data_merged[(data_merged['screen_size'] == "Small")&(data_merged['swipe_finger'] == "Index")]
data_small_other = data_merged[(data_merged['screen_size'] == "Small")&(data_merged['swipe_finger'] == "Other")]

data_big_thumb = data_merged[(data_merged['screen_size'] == "Big")&(data_merged['swipe_finger'] == "Thumb")]
data_big_index = data_merged[(data_merged['screen_size'] == "Big")&(data_merged['swipe_finger'] == "Index")]
data_big_other = data_merged[(data_merged['screen_size'] == "Big")&(data_merged['swipe_finger'] == "Other")]
print(len(data_small_thumb)+ len(data_small_index)+len(data_small_other)+len(data_big_thumb)+len(data_big_index)+len(data_big_other))

1304


In [7]:
#remove outliers by IQR
data_small_thumb = remove_outlier(data_small_thumb, 'error_rate')
data_small_thumb = remove_outlier(data_small_thumb, 'avg_char_duration')

data_small_index = remove_outlier(data_small_index, 'error_rate')
data_small_index = remove_outlier(data_small_index, 'avg_char_duration')

data_small_other = remove_outlier(data_small_other, 'error_rate')
data_small_other = remove_outlier(data_small_other, 'avg_char_duration')

data_small_thumb = remove_outlier(data_small_thumb, 'error_rate')
data_small_thumb = remove_outlier(data_small_thumb, 'avg_char_duration')

data_big_thumb = remove_outlier(data_big_thumb, 'error_rate')
data_big_thumb = remove_outlier(data_big_thumb, 'avg_char_duration')

data_big_index = remove_outlier(data_big_index, 'error_rate')
data_big_index = remove_outlier(data_big_index, 'avg_char_duration')

data_big_other = remove_outlier(data_big_other, 'error_rate')
data_big_other = remove_outlier(data_big_other, 'avg_char_duration')
print(len(data_small_thumb)+ len(data_small_index)+len(data_small_other)+len(data_big_thumb)+len(data_big_index)+len(data_big_other))

964


In [8]:

from pingouin import multivariate_normality
# perform the Multivariate Normality Test
data_small_thumb_test = data_small_thumb[["error_rate", "avg_char_duration"]]

multivariate_normality(data_small_thumb_test, alpha=0.05)

HZResults(hz=6.071437571408224, pval=2.6843549130423245e-13, normal=False)

In [9]:
#shapiro test
print("swipe_finger: Thumb -> error_rate:",stats.shapiro(data_small_thumb['error_rate']))
print("swipe_finger: Index -> error_rate:",stats.shapiro(data_small_index['error_rate']))
print("swipe_finger: Other -> error_rate:",stats.shapiro(data_small_other['error_rate']))

print("swipe_finger: Thumb -> avg_char_duration:",stats.shapiro(data_small_thumb['avg_char_duration']))
print("swipe_finger: Index -> avg_char_duration:",stats.shapiro(data_small_index['avg_char_duration']))
print("swipe_finger: Other -> avg_char_duration:",stats.shapiro(data_small_other['avg_char_duration']))

print("screen_size: Big -> error_rate:",stats.shapiro(data_big_thumb['error_rate']))
print("screen_size: Small -> error_rate:",stats.shapiro(data_big_index['error_rate']))
print("screen_size: Big -> error_rate:",stats.shapiro(data_big_other['error_rate']))

print("screen_size: Big -> avg_char_duration:",stats.shapiro(data_big_thumb['avg_char_duration']))
print("screen_size: Small -> avg_char_duration:",stats.shapiro(data_big_index['avg_char_duration']))
print("screen_size: Big -> avg_char_duration:",stats.shapiro(data_big_other['avg_char_duration']))



swipe_finger: Thumb -> error_rate: ShapiroResult(statistic=0.9082438945770264, pvalue=8.765149196732214e-12)
swipe_finger: Index -> error_rate: ShapiroResult(statistic=0.8906373977661133, pvalue=7.405822799455564e-08)
swipe_finger: Other -> error_rate: ShapiroResult(statistic=0.9083354473114014, pvalue=0.027986247092485428)
swipe_finger: Thumb -> avg_char_duration: ShapiroResult(statistic=0.9441971182823181, pvalue=1.3196005177462666e-08)
swipe_finger: Index -> avg_char_duration: ShapiroResult(statistic=0.8253926038742065, pvalue=1.4631175437873623e-10)
swipe_finger: Other -> avg_char_duration: ShapiroResult(statistic=0.8878774642944336, pvalue=0.010087528266012669)
screen_size: Big -> error_rate: ShapiroResult(statistic=0.9119751453399658, pvalue=9.763157892816315e-14)
screen_size: Small -> error_rate: ShapiroResult(statistic=0.9117352962493896, pvalue=4.315183232961317e-08)
screen_size: Big -> error_rate: ShapiroResult(statistic=0.9100533127784729, pvalue=0.012967057526111603)
screen

In [10]:
merged_df = data_small_thumb.merge(data_small_index, how = 'outer')
merged_df = data_small_other.merge(merged_df, how = 'outer')
merged_df = data_big_thumb.merge(merged_df, how = 'outer')
merged_df = data_big_index.merge(merged_df, how = 'outer')
merged_df = data_big_other.merge(merged_df, how = 'outer')

merged_df

,error_rate,avg_char_duration,screen_size,swipe_finger
0,0.269841,510.376367,Big,Other
1,0.100000,562.493093,Big,Other
2,0.061538,245.970123,Big,Other
3,0.169231,118.374189,Big,Other
4,0.357143,287.704231,Big,Other
...,...,...,...,...
959,0.357143,388.862812,Small,Index
960,0.000000,157.535714,Small,Index
961,0.239437,259.976649,Small,Index
962,0.136364,448.521026,Small,Index


In [14]:
merged_df.to_csv('test_removingoutliers_seperately.csv', sep='\t', encoding='utf-8', index_label=False)
